# City Street Network Orientations / Hungary

Compare the spatial orientations of city streeet networks with OSMnx.
Based on  https://github.com/gboeing/osmnx-examples/blob/master/notebooks/17-street-network-orientations.ipynb

The MIT License (MIT)

Copyright (c) 2017 Geoff Boeing http://geoffboeing.com/

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import osmnx as ox

ox.config(log_console=True, use_cache=True)
weight_by_length = False

ox.__version__

'0.9'

In [2]:
# define the study sites as label : query
places = {'Budapest'      : {'city': 'Budapest', 'country': 'Hungary'},
          'Bugyi'         : {'city': 'Bugyi', 'country': 'Hungary'},
          'Debrecen'      : {'city': 'Debrecen', 'country': 'Hungary'},
          'Szeged'        : {'city': 'Szeged', 'country': 'Hungary'},
          'Miskolc'       : {'city': 'Miskolc', 'country': 'Hungary'},
          'Veszprém'      : {'city': 'Veszprém', 'country': 'Hungary'},
          'Zalaegerszeg'  : {'city': 'Zalaegerszeg', 'country': 'Hungary'},
          'Pilisborosjenő': {'city': 'Pilisborosjenő', 'country': 'Hungary'},
          'Kecskemét'     : {'city': 'Kecskemét', 'country': 'Hungary'},
          'Nagykovácsi'   : {'city': 'Nagykovácsi', 'country': 'Hungary'},
          'Győr'          : {'city': 'Győr', 'country': 'Hungary'},
          'Pécel'         : {'city': 'Pécel', 'country': 'Hungary'}}

In [3]:
# verify OSMnx geocodes each query to what you expect
gdf = ox.gdf_from_places(places.values())
gdf

,bbox_east,bbox_north,bbox_south,bbox_west,geometry,place_name
0,19.334926,47.613147,47.349690,18.925106,"POLYGON ((18.9251057 47.5441443, 18.9297897 47...","Budapest, Central Hungary, Hungary"
1,19.237167,47.299802,47.089159,19.086389,"POLYGON ((19.0863891 47.2738899, 19.0878899 47...","Bugyi, Dabasi járás, Pest megye, Central Hunga..."
2,21.883308,47.636982,47.425047,21.436531,"POLYGON ((21.4365308 47.5355451, 21.4380588 47...","Debrecen, Debreceni járás, Hajdú-Bihar, Northe..."
3,20.308551,46.403952,46.115796,19.972430,"(POLYGON ((19.9724302 46.290371, 19.9773343 46...","Szeged, Szegedi járás, Csongrád, Southern Grea..."
4,20.869835,48.154919,48.023563,20.481677,"POLYGON ((20.4816773 48.0958742, 20.4872067 48...","Miskolc, Miskolci járás, Borsod-Abaúj-Zemplén,..."
5,17.994351,47.205789,47.046295,17.821107,"POLYGON ((17.8211069 47.1813423, 17.8233949 47...","Veszprém, Veszprémi járás, Veszprém, Central T..."
6,16.906932,46.900298,46.780389,16.769437,"POLYGON ((16.7694368 46.8643189, 16.7694692 46...","Zalaegerszeg, Zalaegerszegi járás, Zala, Weste..."
7,19.007671,47.628209,47.587909,18.955607,"POLYGON ((18.9556075 47.6102234, 18.9580276 47...","Pilisborosjenő, Pilisvörösvári járás, Pest meg..."
8,19.851774,47.001254,46.769920,19.506801,"POLYGON ((19.5068013 46.9550725, 19.5095634 46...","Kecskemét, Kecskeméti járás, Bács-Kiskun, Sout..."
9,18.935752,47.596348,47.537567,18.834530,"POLYGON ((18.83453 47.5946883, 18.8357868 47.5...","Nagykovácsi, Budakeszi járás, Pest megye, Cent..."


# Get the street networks and their edge bearings

In [4]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

In [5]:
bearings = {}
for place in sorted(places.keys()):
    
    # get the graph
    query = places[place]
    G = ox.graph_from_place(query, network_type='drive')
    
    # calculate edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    if weight_by_length:
        # weight bearing by length (meters)
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    else:
        # don't weight bearings, just take one value per street segment
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

## Visualize it

In [6]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n*2
    bins = np.arange(n+1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° ans 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [7]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):
    
    bins = np.arange(n+1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width = 2 * np.pi / n
    
    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')
    
    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2, 
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family': 'Century Gothic', 'size':24, 'weight': 'bold'}
    xtick_font = {'family': 'Century Gothic', 'size':10, 'weight': 'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family': 'Century Gothic', 'size':9, 'weight': 'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [8]:
# create figure and axes
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n/ ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# plot each city's polar histogram
for ax, place in zip(axes.flat, sorted(places.keys())):
    polar_plot(ax, bearings[place].dropna(), title=place)
    
# add super title and save full image
suptitle_font = {'family': 'Century Gothic', 'fontsize':60, 'fontweight': 'normal', 'y':1.07}
fig.suptitle('City Street Network Orientation - Hungarian Cities', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig('street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()